## 🧪 2D Cantilever Beam Under Uniform Load — Demonstrating $h$ and Prefinement

Set up a 2D cantilever beam under a **uniform downward traction** on the right edge ($x = L$). The **left edge** ($x = 0$) is clamped. For small deflections, the computed displacement can be compared to the **Euler–Bernoulli beam theory**.

This example is useful for demonstrating the effect of mesh size $h$ and prefinenment techniques.

---

### 📐 Problem Description

- **Geometry**: Rectangular cantilever beam of length $L$ and height $H$
- **Loading**: Uniform downward traction $q$ on the right edge ($x = L$)
- **Boundary Conditions**:
  - Clamped on the left edge ($x = 0$): $\, u = 0$
  - Free on top, bottom, and right edges (except for the traction)

---

### 🧱 Material Properties

- **Given**:
  - Young's modulus: $E = 1000$
  - Poisson's ratio: $\nu = 0.3$

- **Derived**:
  - Shear modulus: $\mu = \dfrac{E}{2(1 + \nu)}$
  - Bulk modulus: $\kappa = \dfrac{E}{3(1 - 2\nu)}$

---

### 🧮 Analytical Solution (Euler–Bernoulli Beam Theory)

The **analytical tip deflection** for a cantilever beam under a **uniform load $P$** in plane strain is given by:

$$
w(L) = -\dfrac{P x^2}{6EI} (3L - x)
$$

This is evaluated at the beam tip ($x = L$) and mid-height ($y = H/2$). The solution serves as a reference for verifying FEM results under mesh refinement or local $h$-refinement near the loaded edge.

---

And using this setup to experiment with mesh density $h$ and investigate how pre-refinement near the tip improves accuracy.



In [1]:
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
# for saving files later
tutorials_dir = Path.cwd()

In [3]:
# --- Beam geometry ---
L = 20.0   # length in x
H = 1.0    # height in y
nx = 80    # number of elements along length
ny = 4     # number of elements along height

# --- FEA problem info --- 
ele_type = "D2_nn8_quad"  # 2D, 4-node quadrilateral (linear)
ndof = 2                  # 2 DOFs per node (x, y)


In [4]:
# Generate a rectangular mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
# coords: shape (n_nodes, 2)
# connect: shape (n_nodes_per_elem, n_elems)

In [5]:
# --- Identify boundaries ---
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

In [6]:
# 1) Clamp the left edge: fix x- and y-displacements = 0
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)

In [7]:
# 2) Uniform downward traction on the top edge (y=H)
# Let q be negative in the y-direction
q = -0.01  # load per unit length in y
# For a 2D plane strain problem, this is a traction (tx, ty) = (0, q)
dload_info = pre.assign_uniform_load_rect(boundary_edges, "right", 0.0, q)

In [8]:
# Combine boundary conditions
fixed_nodes = fixed_left  # only the left edge is clamped

# --- Material properties ---
E = 100000.0
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))

material_props = np.array([mu, kappa])
print(f"Material properties: mu={mu:.3f}, kappa={kappa:.3f}")

Material properties: mu=38461.538, kappa=83333.333


In [9]:
# Number of incremental load steps
nr_num_steps = 1

# --- Solve with your hyperelastic solver ---
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # shape (2, n_nodes)
    connect.T,     # shape (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-10,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # shape: (n_nodes*ndof,)

Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=1.629717e-06, tolerance=1.000000e-10
Iteration 2, Correction=7.875615e-05, Residual=3.140595e-06, tolerance=1.000000e-10
Iteration 3, Correction=1.353098e-12, Residual=7.588358e-14, tolerance=1.000000e-10


In [10]:
# --- Compute the tip displacement from the FEA result ---
# We'll pick a node near x=L, y=H/2
tip_node = None
tol = 1e-3
for i, (x, y) in enumerate(coords):
    if abs(x - L) < tol and abs(y - H/2) < H/(2*ny):
        tip_node = i
        break
if tip_node is None:
    raise ValueError("Could not find tip node near x=L, y=H/2.")

tip_disp_y = final_disp[ndof*tip_node + 1]  # the y-component of displacement

In [11]:
# compute the analytical solution
E_eff = E / (1 - nu ** 2.0)
I = (H**3) / 12.0  # moment of inertia for a rectangular beam
w_analytical = q * L**3 / (3 * E_eff * I)  # analytical solution

In [12]:
print(f"Tip node index: {tip_node}, coordinates={coords[tip_node]}")
print(f"Computed tip deflection (y): {tip_disp_y:.6f}")
print(f"Analytical Euler-Bernoulli deflection: {w_analytical:.6f}")

# --- Evaluate error ---
error = abs(tip_disp_y - w_analytical)
print(f"Absolute error = {error:.6e}")

Tip node index: 644, coordinates=[20.   0.5]
Computed tip deflection (y): -0.002912
Analytical Euler-Bernoulli deflection: -0.002912
Absolute error = 1.225718e-07


In [13]:
# using h-refinement to compute convergence
error = np.array([3.697239e-04, 1.029849e-04, 2.603879e-05,5.726829e-06])
h = np.array([0.5, 0.25, 0.125, 0.0625])

# --- Plotting ---
plt.figure()
plt.plot(h, error, 'o-')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('h (element size)')
plt.ylabel('Error')
plt.title('Convergence of the FEA solution')
plt.grid()
plt.tight_layout()
plt.savefig(tutorials_dir / 'h_convergence_plot.png')


<img src="h_convergence_plot.png" alt="Mesh plot2" style="width:50%;">

In [14]:
# using p-refinement to compute convergence
error = np.array([1.029849e-04, 1.225718e-07])
p = np.array([4, 8])

# --- Plotting ---
plt.figure()
plt.plot(p, error, 'o-')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('p (degree of shape function)')
plt.ylabel('Error')
plt.title('Convergence of the FEA solution')
plt.grid()
plt.tight_layout()
plt.savefig(tutorials_dir / 'p_convergence_plot.png')


<img src="p_convergence_plot.png" alt="Mesh plot2" style="width:50%;">